In [159]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import copy
import pickle
import random
import warnings

import numpy as np

from astropy import units as u
from astropy.table import Table, Column, vstack

import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Gathering the COSMOS QA sample

In [17]:
data_dir = '/Users/song/Dropbox/work/data/merian/qa'

cosmos_dir = '/Users/song/work/cosmos2020/COSMOS2020_R1/'

#### Reference cutout input catalog

In [8]:
ref_cat = Table.read(os.path.join(data_dir, 'g09_broadcut_cosmos-2022-03-21.fits'))

In [9]:
ref_cat[0]

name,prefix,dir,chunk,ra,dec,half_size
,,,,,,arcsec
int64,bytes30,bytes61,int64,float64,float64,float64
43153640956908581,g09_broadcut_43153640956908581,/projects/MERIAN/poststamps/g09_broadcut/1/43153640956908581,1,149.482517432927,1.7997843568838428,3.4999708170211936


#### Key sample: Merian targets - Low redshift dwarfs

- Just using the `ladybug` sample.

In [163]:
dwarf_cat = Table.read(os.path.join(data_dir, 'ladybug_hsc.fits'))

In [164]:
dwarf_new = dwarf_cat['index', 'ra', 'dec', 'z_cosmos'][np.isfinite(dwarf_cat['ra'])]
dwarf_new['name'] = ["ladybug_{:s}".format(str(i)) for i in dwarf_new['index']]
dwarf_new.rename_column('z_cosmos', 'z')
dwarf_new.remove_column('index')
dwarf_new['prefix'] = ["merian_qa_{:s}".format(n) for n in dwarf_new['name']]
dwarf_new['chunk'] = np.full(len(dwarf_new), 1)
dwarf_new['half_size'] = [10.0 * u.arcsec] * len(dwarf_new)
dwarf_new['ra'] = [r * u.deg for r in dwarf_new['ra']]
dwarf_new['dec'] = [r * u.deg for r in dwarf_new['dec']]

In [165]:
dwarf_sub = dwarf_new[np.unique(random.sample(list(np.linspace(0, len(dwarf_new) - 1, dtype='int')), 50))]

In [166]:
_ = [print("{:11s}  {:9.5f}  {:9.5f}".format(obj['name'], obj['ra'], obj['dec'])) for obj in dwarf_sub]

ladybug_201  150.59890    2.67431
ladybug_207  150.69369    2.67780
ladybug_167  149.50354    2.50470
ladybug_219  150.36821    2.70883
ladybug_89   150.01890    2.14322
ladybug_229  150.44217    2.73315
ladybug_17   150.21816    1.84682
ladybug_199  150.44408    2.65832
ladybug_97   149.96707    2.20333
ladybug_50   149.81799    2.00999
ladybug_49   150.42208    2.00509
ladybug_200  150.59221    2.65966
ladybug_78   149.89676    2.08596
ladybug_102  150.62953    2.23930
ladybug_148  149.55791    2.42166
ladybug_147  150.06773    2.42447
ladybug_15   149.60838    1.84373
ladybug_146  150.06704    2.42037
ladybug_159  150.39753    2.47117
ladybug_233  149.92496    2.74792
ladybug_71   149.79650    2.06832
ladybug_30   150.38163    1.93203
ladybug_42   150.12694    1.98492
ladybug_143  149.92566    2.38803
ladybug_61   150.03270    2.04152
ladybug_116  149.65181    2.29403
ladybug_246  150.10670    2.78732
ladybug_129  149.80363    2.33660
ladybug_54   150.17027    2.02762
ladybug_47   1

#### z=0.4 massive galaxies

In [167]:
cos_tab = Table.read(os.path.join(cosmos_dir, 'cosmos2020_farmer_hsc_short.fits'))

In [168]:
selection = (
    (cos_tab['lp_zPDF'] >= 0.40) & (cos_tab['lp_zPDF'] <= 0.45) & 
    (cos_tab['lp_mass_med'] >= 11.0) & (cos_tab['lp_mass_med'] <= 11.8) &
    (cos_tab['SOLUTION_MODEL'] != b'PointSource') &
    (~cos_tab['HSC_i_MAG'].mask)
)

print(selection.sum())

cos_massive = cos_tab[selection]['ID', 'ra', 'dec', 'lp_zPDF'][np.unique(random.sample(list(np.linspace(0, len(cos_tab[selection]) - 1, dtype='int')), 20))]
cos_massive['name'] = ["cos20m_{:s}".format(str(i)) for i in cos_massive['ID']]
cos_massive.rename_column('lp_zPDF', 'z')
cos_massive.remove_column('ID')
cos_massive['prefix'] = ["merian_qa_{:s}".format(n) for n in cos_massive['name']]
cos_massive['chunk'] = np.full(len(cos_massive), 2)
cos_massive['half_size'] = [12.0 * u.arcsec] * len(cos_massive)
cos_massive['ra'] = [r * u.deg for r in cos_massive['ra']]
cos_massive['dec'] = [r * u.deg for r in cos_massive['dec']]

189


In [169]:
_ = [print("{:s}  {:9.5f}  {:9.5f}".format(obj['name'], obj['ra'], obj['dec'])) for obj in cos_massive]

cos20m_22382  150.78264    2.04482
cos20m_87451  149.59568    1.54670
cos20m_101352  149.85681    2.29321
cos20m_191381  149.30678    2.64863
cos20m_216863  149.77611    2.14060
cos20m_302996  150.21909    2.05899
cos20m_367401  149.52547    1.81734
cos20m_384406  149.84340    2.14760
cos20m_431377  149.47430    2.66517
cos20m_502870  150.47400    2.56677
cos20m_506865  149.99692    2.49620
cos20m_525740  150.28220    1.58055
cos20m_553174  149.85845    2.59191
cos20m_567195  150.37773    2.41224
cos20m_726922  149.95640    1.78785
cos20m_805966  150.06947    2.70363
cos20m_832804  149.54468    1.94527
cos20m_854214  149.35099    1.95578
cos20m_937609  150.68245    2.87598
cos20m_964093  150.38681    2.86065


#### z=0.4 star-forming galaxies

In [170]:
selection = (
    (cos_tab['lp_zPDF'] >= 0.40) & (cos_tab['lp_zPDF'] <= 0.42) & 
    (cos_tab['lp_mass_med'] >= 10.4) & (cos_tab['lp_mass_med'] <= 11.0) &
    (cos_tab['SOLUTION_MODEL'] != b'PointSource') &
    (~cos_tab['HSC_i_MAG'].mask) &
    (cos_tab['lp_sSFR_med'] >= -9.8)
)

print(selection.sum())

cos_sf = cos_tab[selection]['ID', 'ra', 'dec', 'lp_zPDF']
cos_sf['name'] = ["cos20s_{:s}".format(str(i)) for i in cos_sf['ID']]
cos_sf.rename_column('lp_zPDF', 'z')
cos_sf.remove_column('ID')
cos_sf['prefix'] = ["merian_qa_{:s}".format(n) for n in cos_sf['name']]
cos_sf['chunk'] = np.full(len(cos_sf), 3)
cos_sf['half_size'] = [12.0 * u.arcsec] * len(cos_sf)
cos_sf['ra'] = [r * u.deg for r in cos_sf['ra']]
cos_sf['dec'] = [r * u.deg for r in cos_sf['dec']]

14


In [171]:
_ = [print("{:s}  {:9.5f}  {:9.5f}".format(obj['name'], obj['ra'], obj['dec'])) for obj in cos_sf]

cos20s_112607  150.71982    1.96539
cos20s_172077  149.32428    1.56284
cos20s_183011  150.64267    2.80175
cos20s_248630  149.33251    2.81992
cos20s_254486  149.91868    2.47723
cos20s_300506  150.46214    2.06322
cos20s_360480  149.65309    1.66436
cos20s_390609  150.22851    2.52392
cos20s_436814  150.77965    2.09753
cos20s_597093  149.85381    2.24566
cos20s_689919  149.73084    2.34368
cos20s_837716  150.01288    2.85205
cos20s_855161  150.32796    2.11899
cos20s_914463  150.41554    2.77689


#### Low-z SF galaxies

In [172]:
selection = (
    (cos_tab['lp_zPDF'] >= 0.05) & (cos_tab['lp_zPDF'] <= 0.12) & 
    (cos_tab['lp_mass_med'] >= 10.0) & (cos_tab['lp_mass_med'] <= 10.5) &
    (cos_tab['SOLUTION_MODEL'] != b'PointSource') &
    (~cos_tab['HSC_i_MAG'].mask) & 
    (~cos_tab['HSC_g_MAG'].mask) &
    (~cos_tab['HSC_r_MAG'].mask) &
    (~cos_tab['HSC_z_MAG'].mask) &
    (cos_tab['lp_sSFR_med'] >= -11.8)
)

print(selection.sum())

cos_lowz = cos_tab[selection]['ID', 'ra', 'dec', 'lp_zPDF']
cos_lowz['name'] = ["cos20l_{:s}".format(str(i)) for i in cos_lowz['ID']]
cos_lowz.rename_column('lp_zPDF', 'z')
cos_lowz.remove_column('ID')
cos_lowz['prefix'] = ["merian_qa_{:s}".format(n) for n in cos_lowz['name']]
cos_lowz['chunk'] = np.full(len(cos_lowz), 4)
cos_lowz['half_size'] = [20.0 * u.arcsec] * len(cos_lowz)
cos_lowz['ra'] = [r * u.deg for r in cos_lowz['ra']]
cos_lowz['dec'] = [r * u.deg for r in cos_lowz['dec']]

_ = [print("{:s}  {:9.5f}  {:9.5f}".format(obj['name'], obj['ra'], obj['dec'])) for obj in cos_lowz]

22
cos20l_26603  149.29809    2.54458
cos20l_86655  149.71681    2.47782
cos20l_97320  149.61410    2.29873
cos20l_197293  150.58901    2.13695
cos20l_212576  150.48801    2.05326
cos20l_316247  149.42215    2.56843
cos20l_333542  149.23653    1.98479
cos20l_339733  149.62324    2.32156
cos20l_387135  149.76919    2.82353
cos20l_391890  149.41851    2.07138
cos20l_399866  149.94515    2.82042
cos20l_407572  149.78743    2.67219
cos20l_469216  149.97716    2.00485
cos20l_471731  149.70700    2.15898
cos20l_559872  150.37942    2.48939
cos20l_579080  149.37761    2.44471
cos20l_728535  150.13885    2.52316
cos20l_752726  149.71828    2.85676
cos20l_834597  149.92841    2.59209
cos20l_849157  150.40551    1.61906
cos20l_928084  150.68130    2.20619
cos20l_964460  150.08213    2.87683


In [173]:
index_use = [1, 8, 12, 18, 20]

cos_lowz = cos_lowz[index_use]

In [174]:
_ = [print("{:s}  {:9.5f}  {:9.5f}".format(obj['name'], obj['ra'], obj['dec'])) for obj in cos_lowz]

cos20l_86655  149.71681    2.47782
cos20l_387135  149.76919    2.82353
cos20l_469216  149.97716    2.00485
cos20l_834597  149.92841    2.59209
cos20l_928084  150.68130    2.20619


#### Bright Stars

In [175]:
ps1_tab = Table.read(os.path.join(data_dir, 'cosmos_ps1_refcat.fits'))

In [176]:
selection = (
    (np.array(ps1_tab['MEDIAN'])[:, 4] <= 13.5) & 
    (np.array(ps1_tab['MEDIAN'])[:, 4] >= 11.0)
)

print(selection.sum())

ps1_use = ps1_tab[selection]['OBJ_ID', 'RA', 'DEC']

ps1_use['name'] = ["ps1_{:s}".format(str(i)) for i in ps1_use['OBJ_ID']]
ps1_use.rename_column('RA', 'ra')
ps1_use.rename_column('DEC', 'dec')
ps1_use.remove_column('OBJ_ID')
ps1_use['prefix'] = ["merian_qa_{:s}".format(n) for n in ps1_use['name']]
ps1_use['chunk'] = np.full(len(ps1_use), 5)
ps1_use['z'] = np.full(len(ps1_use), 0)
ps1_use['half_size'] = [25.0 * u.arcsec] * len(ps1_use)
ps1_use['ra'] = [r * u.deg for r in ps1_use['ra']]
ps1_use['dec'] = [r * u.deg for r in ps1_use['dec']]

ps1_use = ps1_use[np.unique(random.sample(list(np.linspace(0, len(ps1_use) - 1, dtype='int')), 15))]

_ = [print("{:s}  {:9.5f}  {:9.5f}".format(obj['name'], obj['ra'], obj['dec'])) for obj in ps1_use]

215
ps1_3921720481765406300  150.86029    2.03471
ps1_3939682103717201378  151.19408    2.58117
ps1_4011757289941519439  150.16748    2.98027
ps1_4029754096264348796  150.32932    3.33935
ps1_3831666081404417370  151.49728    0.99505
ps1_3903794044186137847  150.06874    1.05334
ps1_3939858025577208529  149.50625    1.09571
ps1_3975834046038042355  149.45330    1.78451
ps1_3993918813291715632  148.72393    1.40335
ps1_4029894833755034027  148.97017    2.10587
ps1_4047856455703869211  149.21851    2.60695
ps1_4065853262027313982  149.25266    2.98728
ps1_4083850068350751573  149.12964    3.24068
ps1_4101864466860233459  149.06548    3.29661
ps1_3957661317854604031  151.32683    2.85437


#### High-redshift Lyman alpha nebulae from Mingyu Li

In [3]:
lya_cat = Table.read(os.path.join(data_dir, 'cosmos_highz_lya_nabulae.csv'))

In [4]:
lya_cat

Name,RA,DEC,LyaWave,z
str6,str13,str14,str7,str6
z33-0,10:01:27.4792,+02:08:35.8271,5270?,3.3?
HPS168,10:00:09.69,+02:14:54.9,5409.27,3.4496
z49-0,10:00:38.3805,+02:11:27.6213,7150?,4.9?
z49-1,10:01:45.99,+2:02:43.86,7152,4.888


In [153]:
obj_1 = {
    'name': 'lya_z330', 'ra': 150.3644967 * u.deg, 'dec': 2.1432853 * u.deg, 'z': 3.3, 'half_size': 5.0 * u.arcsec,
    'prefix': 'merian_qa_lya_z330', 'chunk': 6
}

obj_2 = {
    'name': 'lya_s168', 'ra': 150.0403750 * u.deg, 'dec': 2.2485833 * u.deg, 'z': 3.4, 'half_size': 5.0 * u.arcsec,
    'prefix': 'merian_qa_lya_s168', 'chunk': 6
}

obj_3 = {
    'name': 'lya_z490', 'ra': 150.1599188 * u.deg, 'dec': 2.1910059 * u.deg, 'z': 4.9, 'half_size': 5.0 * u.arcsec,
    'prefix': 'merian_qa_lya_z490', 'chunk': 6
}

obj_4 = {
    'name': 'lya_z491', 'ra': 150.4416250 * u.deg, 'dec': 2.0455167 * u.deg, 'z': 4.9, 'half_size': 5.0 * u.arcsec,
    'prefix': 'merian_qa_lya_z491', 'chunk': 6
}

lya_tab = Table([obj_1, obj_2, obj_3, obj_4])

In [154]:
_ = [print("{:s}  {:8.4f}  {:8.4f}".format(obj['name'], obj['ra'], obj['dec'])) for obj in lya_tab]

lya_z330  150.3645    2.1433
lya_s168  150.0404    2.2486
lya_z490  150.1599    2.1910
lya_z491  150.4416    2.0455


In [155]:
lya_tab

name,ra,dec,z,half_size,prefix,chunk
,deg,deg,,arcsec,,
str8,float64,float64,float64,float64,str18,int64
lya_z330,150.3644967,2.1432853,3.3,5.0,merian_qa_lya_z330,6
lya_s168,150.040375,2.2485833,3.4,5.0,merian_qa_lya_s168,6
lya_z490,150.1599188,2.1910059,4.9,5.0,merian_qa_lya_z490,6
lya_z491,150.441625,2.0455167,4.9,5.0,merian_qa_lya_z491,6


In [177]:
merian_qa_sample = vstack([dwarf_sub, cos_massive, cos_sf, cos_lowz, ps1_use, lya_tab])

In [179]:
merian_qa_sample.write(os.path.join(data_dir, 'merian_qa_20220419.fits'), overwrite=True)